In [46]:
from glob import glob
from PIL import Image
import imagehash

In [47]:
files = sorted(glob('cropped/*.png'))
files[:5]

['cropped/000.png',
 'cropped/001.png',
 'cropped/002.png',
 'cropped/003.png',
 'cropped/004.png']

In [ ]:
MIN_FRAMES = 5

In [48]:
stack = []

for i in range(len(files) - 1):
    prev_f = files[i]
    next_f = files[i + 1]
    
    prev_h = imagehash.average_hash(Image.open(prev_f))
    next_h = imagehash.average_hash(Image.open(next_f))

    if next_h == prev_h:
        stack.append(i)
        print("adding", i)

    if next_h != prev_h:
        print(stack)
        stack = []

adding 2
adding 3
adding 5
adding 26
adding 28
adding 33
[2, 3, 5, 26, 28, 33]
adding 40
adding 65
adding 76
adding 85
adding 90
adding 91
[40, 65, 76, 85, 90, 91]
adding 94
adding 103
adding 104
adding 105
adding 110
adding 113
[94, 103, 104, 105, 110, 113]
adding 115
adding 116
adding 118
adding 123
adding 125
adding 135
[115, 116, 118, 123, 125, 135]
adding 137
adding 138
adding 140
adding 142
adding 144
adding 145
[137, 138, 140, 142, 144, 145]
adding 147
adding 148
adding 149
adding 150
adding 151
adding 152
adding 153
adding 154
adding 155
[147, 148, 149, 150, 151, 152, 153, 154, 155]
adding 157
adding 159
adding 161
adding 162
adding 163
adding 164
adding 165
[157, 159, 161, 162, 163, 164, 165]
adding 167
adding 173
adding 175
adding 176
adding 177
adding 178
adding 179
adding 180
[167, 173, 175, 176, 177, 178, 179, 180]
adding 183
adding 184
adding 186
adding 189
adding 191
adding 192
[183, 184, 186, 189, 191, 192]
adding 196
adding 197
adding 199
adding 200
adding 201
adding 2

In [89]:
ss = [2, 3, 5, 26, 28, 33]

nn = []
for i in range(len(ss) - 1):
    p = ss[i]
    n = ss[i + 1]
    if n - p < 3:
        nn.append(p)
if n - p < 3:
    nn.append(n)
nn

[2, 3, 26]

In [87]:
def tight(ss):
    nn = []
    for i in range(len(ss) - 1):
        p = ss[i]
        n = ss[i + 1]
        if n - p < 3:
            nn.append(p)
    if n - p < 3:
        nn.append(n)
    return n